In [1]:
import numpy as np
import torch
from torch import nn
import matplotlib.pyplot as plt

In [2]:
PROBES_DIM_X = 50
PROBES_DIM_Y = 5
PROBES_DIM_Z = 5
PROBES_COUNT = PROBES_DIM_X * PROBES_DIM_Y * PROBES_DIM_Z

INPUT_FLOAT_COUNT = PROBES_COUNT + 18 # probe_feature + pos + angle
SH_FLOAT_COUNT = 27
MLP_HIDDEN_LAYER_WIDTH = 12

In [3]:
class NeuralLightmap(nn.Module):
    def __init__(self):
        super(NeuralLightmap, self).__init__()
        self.probe_features = nn.Parameter(torch.rand(PROBES_COUNT), requires_grad=True)

        self.hidden_layer = nn.Linear(INPUT_FLOAT_COUNT, MLP_HIDDEN_LAYER_WIDTH)
        self.output_layer = nn.Linear(MLP_HIDDEN_LAYER_WIDTH, SH_FLOAT_COUNT)

    def trigonometric_encoding(self, x: float, L: int):
        assert x.ndim == 2
        y = []
        for i in range(L):
            s = torch.sin(2**i * torch.pi * x)
            c = torch.cos(2**i * torch.pi * x)
            y.append(s)
            y.append(c)
        y = torch.cat(y, dim=1)
        return y

    def forward(self, pos, angle):
        pos = pos.view(-1, 3)
        angle = angle.view(-1, 1)

        assert pos.shape[0] == angle.shape[0]
        batch_size = pos.shape[0]

        pos_enc = self.trigonometric_encoding(pos, L=2)
        angle_enc = self.trigonometric_encoding(angle, L=3)

        x = torch.cat([pos_enc, angle_enc, self.probe_features.repeat(batch_size, 1)], dim=1)
        x = self.hidden_layer(x)
        x = torch.sigmoid(x)
        x = self.output_layer(x)
        return x


In [4]:
xx, yy, zz = np.meshgrid(
    np.linspace(0.0, 1.0, PROBES_DIM_X),
    np.linspace(0.0, 1.0, PROBES_DIM_Y),
    np.linspace(0.0, 1.0, PROBES_DIM_Z),
)
xx = xx.reshape(-1, 1)
yy = yy.reshape(-1, 1)
zz = zz.reshape(-1, 1)
pos_grid = np.hstack([xx, yy, zz])

In [5]:
light_angle = 0.0

In [6]:
from pathlib import Path

from texture_sampler import Texture, load_texture_by_name, sample_uv

DATA_DIR = Path("LightmapsData/")


def Texture3DSample(tex: Texture, uvw: np.ndarray):
    uvw = uvw.reshape(-1, 3)
    _, _, _, channels = tex.data.shape
    results = []
    for p in uvw:
        color = sample_uv(tex, p[0], p[1], p[2])
        color = color.reshape(-1, channels)
        results.append(color)
    results = np.concatenate(results, axis=0)
    return results


def GetVolumetricLightmapAmbient(BrickTextureUVs: np.ndarray):
    tex = load_texture_by_name(DATA_DIR, "AmbientVector")
    return Texture3DSample(tex, BrickTextureUVs)


def GetVolumetricLightmapSHCoefficients0(BrickTextureUVs: np.ndarray):
    AmbientVector = GetVolumetricLightmapAmbient(BrickTextureUVs)
    SHCoefficients0Red = Texture3DSample(load_texture_by_name(DATA_DIR, "SHCoefficients_0"), BrickTextureUVs) * 2 - 1
    SHCoefficients0Green = Texture3DSample(load_texture_by_name(DATA_DIR, "SHCoefficients_2"), BrickTextureUVs) * 2 - 1
    SHCoefficients0Blue = Texture3DSample(load_texture_by_name(DATA_DIR, "SHCoefficients_4"), BrickTextureUVs) * 2 - 1
    SHDenormalizationScales0 = np.array([
        0.488603 / 0.282095,
		0.488603 / 0.282095,
		0.488603 / 0.282095,
		1.092548 / 0.282095
    ])
    SHCoefficients0Red = SHCoefficients0Red * AmbientVector[:, 0:1] * SHDenormalizationScales0
    SHCoefficients0Green = SHCoefficients0Green * AmbientVector[:, 1:2] * SHDenormalizationScales0
    SHCoefficients0Blue = SHCoefficients0Blue * AmbientVector[:, 2:3] * SHDenormalizationScales0
    return AmbientVector, SHCoefficients0Red, SHCoefficients0Green, SHCoefficients0Blue


def GetVolumetricLightmapSH3(BrickTextureUVs: np.ndarray):
    AmbientVector, SHCoefficients0Red, SHCoefficients0Green, SHCoefficients0Blue = GetVolumetricLightmapSHCoefficients0(BrickTextureUVs)
    SHCoefficients1Red = Texture3DSample(load_texture_by_name(DATA_DIR, "SHCoefficients_1"), BrickTextureUVs) * 2 - 1
    SHCoefficients1Green = Texture3DSample(load_texture_by_name(DATA_DIR, "SHCoefficients_3"), BrickTextureUVs) * 2 - 1
    SHCoefficients1Blue = Texture3DSample(load_texture_by_name(DATA_DIR, "SHCoefficients_5"), BrickTextureUVs) * 2 - 1
    SHDenormalizationScales1 = np.array([
		1.092548 / 0.282095,
		4.0 * 0.315392 / 0.282095,
		1.092548 / 0.282095,
		2.0 * 0.546274 / 0.282095
    ])
    SHCoefficients1Red = SHCoefficients1Red * AmbientVector[:, 0:1] * SHDenormalizationScales1
    SHCoefficients1Green = SHCoefficients1Green * AmbientVector[:, 1:2] * SHDenormalizationScales1
    SHCoefficients1Blue = SHCoefficients1Blue * AmbientVector[:, 2:3] * SHDenormalizationScales1

    IrradianceSH = np.concatenate([
        AmbientVector[:, 0:1], # .x
        SHCoefficients0Red[:],
        SHCoefficients1Red[:],
        AmbientVector[:, 1:2], # .y
        SHCoefficients0Green[:],
        SHCoefficients1Green[:],
        AmbientVector[:, 2:3], # .z
        SHCoefficients0Blue[:],
        SHCoefficients1Blue[:],
    ], axis=1)
    return IrradianceSH

In [7]:
# BrickTextureUVs __i06676.x, __i16677.x, __i26678.x float3 0.31288, 0.43666, 0.66076

# V0 _8422.x, __i06913.x, __i16915.x, __i26917.x float4 1.48535, -0.41522, 0.71059, 0.04087
# V1 __i36919.x, __i06997.x, __i16999.x, __i27001.x float4 -0.1588, -0.80276, 0.53671, 0.2306
# V2 __i37003.x float -0.3305
# V0 _8423.x, __i06925.x, __i16927.x, __i26929.x float4 0.98926, -0.75354, 0.73377, 0.08955
# V1 __i36931.x, __i07009.x, __i17011.x, __i27013.x float4 -0.16142, -0.97288, 0.60831, 0.24069
# V2 __i37015.x float -0.27203
# V0 _8424.x, __i06937.x, __i16939.x, __i26941.x float4 0.81299, -0.61416, 0.62958, 0.08644
# V1 __i36943.x, __i07021.x, __i17023.x, __i27025.x float4 -0.1487, -0.83557, 0.52588, 0.21856
# V2 __i37027.x float -0.2175


In [8]:
shader_value = np.array([
    1.48535, -0.41522, 0.71059, 0.04087,
    -0.1588, -0.80276, 0.53671, 0.2306,
    -0.3305,
    0.98926, -0.75354, 0.73377, 0.08955,
    -0.16142, -0.97288, 0.60831, 0.24069,
    -0.27203,
    0.81299, -0.61416, 0.62958, 0.08644,
    -0.1487, -0.83557, 0.52588, 0.21856,
    -0.2175,
])

this_value = GetVolumetricLightmapSH3(np.array([0.31288, 0.43666, 0.66076]).reshape(1, 3))

print("shader: ", shader_value)
print("this: ", this_value)
print(f"max abs error: {np.abs(shader_value - this_value).max():.5f}")

shader:  [ 1.48535 -0.41522  0.71059  0.04087 -0.1588  -0.80276  0.53671  0.2306
 -0.3305   0.98926 -0.75354  0.73377  0.08955 -0.16142 -0.97288  0.60831
  0.24069 -0.27203  0.81299 -0.61416  0.62958  0.08644 -0.1487  -0.83557
  0.52588  0.21856 -0.2175 ]
this:  [[ 1.48513877 -0.41543918  0.71072756  0.04014002 -0.15832449 -0.80275309
   0.53758384  0.22957705 -0.33113261  0.9894107  -0.75399901  0.73368275
   0.0887034  -0.1607023  -0.97259599  0.60830068  0.24001047 -0.27270406
   0.81303811 -0.61448763  0.62945858  0.08567148 -0.14806844 -0.83529595
   0.52589594  0.21794688 -0.21810566]]
max abs error: 0.00102
